In [99]:
import pandas as pd
import pickle
import csv
import os
import re
import emoji
from textblob import TextBlob
import nltk
from langdetect import detect
nltk.download('punkt')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics import confusion_matrix
import gensim
nltk.download('vader_lexicon')
from sklearn.feature_extraction.text import TfidfVectorizer
import string
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib import cm
import math
from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\luoyu\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [62]:
## Cleaning methods
def clean_emoji(text):
    text=emoji.demojize(text)
    return text
def remove_url(text):
    text = ' '.join(re.sub("http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"," ", text).split())
    return text
def remove_symbol(text):
    text = str.replace(text,',',' ')
    text = str.replace(text,'+',' ')
    text = str.replace(text,'=',' ')
    text = ' '.join(re.sub("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})"," ", text).split())
    text = str.replace(text,"'",' ')
    text = str.replace(text,'"',' ')
    text = str.replace(text,'!',' ')
    text = str.replace(text,'^',' ')
    text = str.replace(text,'(',' ')
    text = str.replace(text,')',' ')
    text = str.replace(text,'%',' ')
    text = str.replace(text,'-',' ')
    text = str.replace(text,'_',' ')
    text = str.replace(text,'|',' ')
    text = str.replace(text,'.',' ')
    text = str.replace(text,':',' ')
    return text    
## this is the optional function we can use to remove @ and other. Might not use it if we need to analyze retweets
def optional_rm(text):
    all_words = text.split(' ')
    remove_words = []
    for each in all_words:
        if each == ' ' or each == '':
            continue
        if each[0] == '#' or each[0] == '@':
            remove_words.append(each)
            if each[0] == '#':
                hashtags = each.split('#')
                for tag in hashtags:
                    tag = re.sub(r'[^\w\s]','',tag)
                    gl_hashtag.append(tag.lower())

    for word in remove_words:
        all_words.remove(word)
    text = ' '.join(all_words)
    
    text = text.split(' ')
    new_text = []
    for each in text:
        if(str.find(each,'http') != -1):
            continue
        
        if not each.isalnum():
            continue
        new_text.append(str.lower(each));
    text = ' '.join(new_text)
    
    return text


def is_english(text):
    try:
        if detect(text) == "en":
            return True
        else:
            return False
    except Exception as e:
        return True
        print(e)   
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True    
def processing(text):
    #text=clean_emoji(text)
    #text=remove_url(text)
    #text = remove_symbol(text)
    text = optional_rm(text)
    return text
## Import Data
def read_json_to_dataframe(filename):
    print(filename)
    #f = open(filename,encoding = 'utf-16').read()
    f1 = pd.read_json(filename, orient='records',lines=True,encoding = 'utf-16')
    return f1

In [220]:
##read files
def read_in(path):
    files = os.listdir(path)
    all_dfs = []
    #print(files)
    for i in files:
        if 'json' in i:
            each_df = read_json_to_dataframe(path+i)
            all_dfs.append(each_df)
            
    return pd.concat(all_dfs, ignore_index=True)

all_df = read_in('C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/')
all_df.shape

C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-19-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-20-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-21-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-22-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-23-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-24-2020.json
C:/Users/luoyu/Desktop/USCISI/vaccine hesitancy/raw/10-25-2020.json


(371940, 23)

In [221]:
## Get rid of the folded tweets
test_df = all_df.copy()
test_df=test_df[~test_df['og_tweet_txt'].str.endswith('\u2026',na=True)]
test_df=test_df[~((test_df['tweet_txt'].str.endswith('\u2026',na=True)) & (test_df['og_tweet_txt'].str.strip() == ""))]
test_df = test_df.drop_duplicates(subset=['tweet_txt'])
# test_df=test_df.dropna(axis=0,how='any')

In [222]:
def clean_txt(df):
    #df["og_tweet_txt"] = df['og_tweet_txt'].apply(lambda x: processing(x))
    df['tweet_txt'] = df['tweet_txt'].apply(lambda x: processing(x))
    #df[df["tweet_txt"].apply(is_english)]
    #df[df["og_tweet_txt"].apply(is_english)]
    return df

In [87]:
global gl_hashtag
gl_hashtag = []
test_copy = test_df.copy()
clean_df = clean_txt(test_copy)
stop_hash_word = ['','covidvaccine','corona','covax','covidvaccination','covid19vaccination','coronavirus','郭文贵\n','爆料革命','pandemic','nhs','7news','covidvaccines','郭文贵','covid19','breaking:','health','covid19\nhttpstcoi6p9ute11f','vaccination','news','covid19\n','covid19vaccine','vaccines,','covidー19','breaking','covid-19','vaccine','vaccines','covid__19','covid_19','covid19,','coronavirus','covid','covid-19','china\n\nhttps://t.co/mg6dph4xp7','edappadipalaniswami', 'bo…']
for word in stop_hash_word:
        gl_hashtag = list(filter(lambda a: a != word, gl_hashtag))
df_tags = pd.DataFrame(gl_hashtag,columns=['hashtag'])
print(len(df_tags['hashtag'].unique()))
print(len(df_tags))
df_count = df_tags.groupby(['hashtag']).size().reset_index(name='counts')
top_hashtags_old = df_count.sort_values(by='counts',ascending = False).head(30)

22939
89188


### keyword comaprision with tfidf

In [168]:
with open('./common english word 1000.txt') as f:
    content = f.readlines()
stopword = [x.strip() for x in content] 
with open('./stopword.txt') as f:
    content = f.readlines()
stopword.extend([x.strip() for x in content])
stopword_more=['getting', 'going','like','think','ever','still','make','one','could','even','take','people','good','says','developed','get','may',"i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]
stopword.extend(stopword_more)
stop_hash_word = ['weeks','county', 'didn', 'doesn', 'hasn', 'haven', 'isn', 'keeps', 'shouldn', 'tv', 'days','amp','000','ll','ve','mr', 'mrs', 'pm', 'dr', 'vaccines','year','rt','covid19','virus','via','covid-19','vaccine','coronavirus','covid','covid-19','china\n\nhttps://t.co/mg6dph4xp7','edappadipalaniswami', 'bo…','19', 'bo', 'china', 'co', 'https', 'mg6dph4xp7']
stopword.extend(stop_hash_word)
stopword = list(set(stopword))

In [223]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm='l2',use_idf=True,max_features=5000,stop_words=stopword,ngram_range=(1,2))
tfIdf = vectorizer.fit_transform(test_df['tweet_txt'])

C:\Users\luoyu\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['al', 'american', 'congress', 'democrat', 'republican'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


In [224]:
tfIdf

<128472x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 560913 stored elements in Compressed Sparse Row format>

In [225]:
size = 128472
vocabs = vectorizer.get_feature_names()
row_keys=[]
for i in range(size):
    row_keys.append([vocabs[i] for i in (tfIdf[i]).nonzero()[1]])
test_df['keywords'] = row_keys

In [226]:
class_keywords = [item for sublist in list(test_df['keywords']) for item in sublist]
top_keywords = [(key,"{0:.2f}%".format(value/size * 100)) for key, value in Counter(class_keywords).most_common()]

In [219]:
new_data_dict = Counter(class_keywords)
new_data_df = pd.DataFrame(top_keywords)

In [227]:
old_data_dict = Counter(class_keywords)
old_data_df = pd.DataFrame(top_keywords)

In [228]:
new_data_df = new_data_df.rename(columns={0: "keyword", 1: "percentage newdata"})
old_data_df = old_data_df.rename(columns={0: "keyword", 1: "percentage olddata"})

In [273]:
joined = old_data_df.set_index('keyword').join(new_data_df.set_index('keyword'), on='keyword',how='inner')
joined['percentage newdata'] = joined['percentage newdata'].str.rstrip('%').astype('float')/100
joined['percentage olddata'] = joined['percentage olddata'].str.rstrip('%').astype('float')/100

In [274]:
joined['difference'] = abs(joined['percentage newdata'] - joined['percentage olddata'])
joined['old_to_new_incease'] = np.where(joined['percentage newdata']-joined['percentage olddata'] > 0,1,0) 

In [275]:
joined_sorted = joined.sort_values(by=['difference'],ascending=False)
#joined_sorted.to_excel('./old_new_data_comparision_full_2531.xlsx')

In [317]:
joined.index = joined.index.astype('str')
search_joined = joined[joined.index.str.contains('mask')]
search_joined.append(search_joined.sum().rename('Total'))

,percentage olddata,percentage newdata,difference,old_to_new_incease
keyword,,,,
mask,0.0066,0.0088,0.0022,1.0
masks,0.0056,0.0062,0.0006,1.0
wearing masks,0.0007,0.0009,0.0002,1.0
wearing mask,0.0007,0.0011,0.0004,1.0
masks distancing,0.0004,0.0005,0.0001,1.0
mask wearing,0.0003,0.0005,0.0002,1.0
mask distance,0.0003,0.0004,0.0001,1.0
masking,0.0002,0.0005,0.0003,1.0
Total,0.0148,0.0189,0.0041,8.0


In [316]:
search_joined.index

Index(['antivax', 'antivaxxers'], dtype='object', name='keyword')